In [1]:
# Install dependencies (run this once in the notebook)
%pip install pandas scikit-learn nltk openpyxl --quiet


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Import libraries
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import nltk
from nltk.corpus import stopwords

# Download NLTK stopwords
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rford\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [3]:
# Keywords to KEEP
keep_dept_keywords = [
    "Planning", "Community Development", "Building Department", "Code Compliance",
    "Public Works", "Streets", "Engineering", "Construction", "Zoning"
]

keep_position_keywords = [
    "Construction", "Building", "HVAC", "Plans Inspector", "Zoning Investigator",
    "Architect", "Permit Technician", "Safety Inspector", "Planner", "Code Enforcer",
    "Plan Check Coordinator", "Licensing Specialist", "Community Development Technician"
]

# Keywords to FILTER OUT
filter_dept_keywords = [
    "Administration", "Communications", "Animal", "Finance", "Community Services",
    "Municipal Power", "Base Reuse", "City Attorney", "City Clerk", "Council", "Fire",
    "Human Resources", "Library", "IT", "Police", "Recreation", "Government Services",
    "Facilities", "Water", "Power", "Airport", "Health", "Zero Waste", "Sewer",
    "Equipment", "Transfer Station", "Janitor", "Landscaping", "Utilities", "Business",
    "Transit", "Nutrition", "Aging", "Adult", "Youth", "Homeless", "Athletics", "Law",
    "Attorney", "Management"
]

filter_position_keywords = [
    "Fitness instructor", "Recreational Leader", "Librarian", "Admin", "Clerk",
    "Secretary", "Parking Officer", "Police", "Firefighter", "Treasurer", "Employment Worker",
    "Lifeguard", "Animal", "Recycling", "Sewer", "Finance", "HR", "IT", "Parks & Rec",
    "Health", "EMT", "Custodian", "Equipment Mechanic", "Aquatics", "Traffic",
    "Community Service", "Veterinary", "Senior Citizen", "Pool", "Utilities", "Audio",
    "Communications", "Mayor", "Attendant", "Vocational Worker", "Messenger Clerk",
    "Customer Service", "Truck Operator", "Sanitation", "Painter", "Eltl Engr Assoc",
    "Laborer", "Program Assistant"
]


In [4]:
# Helper function to match keywords
def keyword_match(text, keywords):
    if pd.isna(text):
        return False
    return any(re.search(rf'\b{re.escape(k)}\b', str(text), re.IGNORECASE) for k in keywords)

# Label rows for training
def label_row(row):
    keep_dept = keyword_match(row['DepartmentOrSubdivision'], keep_dept_keywords)
    keep_pos = keyword_match(row['Position'], keep_position_keywords)
    filter_dept = keyword_match(row['DepartmentOrSubdivision'], filter_dept_keywords)
    filter_pos = keyword_match(row['Position'], filter_position_keywords)
    return int((keep_dept or keep_pos) and not (filter_dept or filter_pos))


In [5]:
# Load training data
train_df = pd.read_excel("city_training_data.xlsx")

# Apply labeling
train_df["label"] = train_df.apply(label_row, axis=1)

# Combine text for model training
train_df["combined_text"] = train_df["DepartmentOrSubdivision"].fillna('') + " " + train_df["Position"].fillna('')

# Show label distribution
train_df["label"].value_counts()


label
0    91664
1    11646
Name: count, dtype: int64

In [6]:
# Feature & target
X = train_df["combined_text"]
y = train_df["label"]

# Create pipeline with TF-IDF + Logistic Regression
model = Pipeline([
    ("tfidf", TfidfVectorizer(stop_words=stopwords.words('english'), max_features=1000)),
    ("clf", LogisticRegression(solver='liblinear', random_state=42))
])

# Train model
model.fit(X, y)


,steps,"[('tfidf', ...), ('clf', ...)]"
,transform_input,None
,memory,None
,verbose,False
,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None


In [7]:
# Load test data
test_df = pd.read_excel("city_test_data.xlsx")

# Combine text columns
test_df["combined_text"] = test_df["DepartmentOrSubdivision"].fillna('') + " " + test_df["Position"].fillna('')

# Predict using the trained model
test_df["predicted_label"] = model.predict(test_df["combined_text"])

# Filter the results
filtered_test_df = test_df[test_df["predicted_label"] == 1]

# Display summary
print(f"Filtered rows: {len(filtered_test_df)} / {len(test_df)}")
filtered_test_df.head()


Filtered rows: 27248 / 241056


,Year,EmployerType,EmployerName,DepartmentOrSubdivision,Position,ElectedOfficial,Judicial,OtherPositions,MinPositionSalary,MaxPositionSalary,...,PensionFormula,EmployerURL,EmployerPopulation,LastUpdatedDate,EmployerCounty,SpecialDistrictActivities,IncludesUnfundedLiability,SpecialDistrictType,combined_text,predicted_label
10,2023,City,Adelanto,Community Services,Planning Commissioner,False,False,NaN,100.0,2400.0,...,NaN,https://www.ci.adelanto.ca.us/198/Human-Resources,36131,2024-06-25,San Bernardino,NaN,False,NaN,Community Services Planning Commissioner,1
11,2023,City,Adelanto,Community Services,Planning Commissioner,False,False,NaN,100.0,2400.0,...,NaN,https://www.ci.adelanto.ca.us/198/Human-Resources,36131,2024-06-25,San Bernardino,NaN,False,NaN,Community Services Planning Commissioner,1
12,2023,City,Adelanto,Community Services,Planning Commissioner,False,False,NaN,100.0,2400.0,...,NaN,https://www.ci.adelanto.ca.us/198/Human-Resources,36131,2024-06-25,San Bernardino,NaN,False,NaN,Community Services Planning Commissioner,1
13,2023,City,Adelanto,Community Services,Planning Commissioner,False,False,NaN,100.0,2400.0,...,NaN,https://www.ci.adelanto.ca.us/198/Human-Resources,36131,2024-06-25,San Bernardino,NaN,False,NaN,Community Services Planning Commissioner,1
20,2023,City,Adelanto,Streets,Maint Worker I,False,False,NaN,44363.0,49931.0,...,2%@62,https://www.ci.adelanto.ca.us/198/Human-Resources,36131,2024-06-25,San Bernardino,NaN,False,NaN,Streets Maint Worker I,1


In [8]:
# Save the filtered test data
filtered_test_df.to_excel("filtered_test_data.xlsx", index=False)
print("✅ Filtered test data saved as 'filtered_test_data.xlsx'")


✅ Filtered test data saved as 'filtered_test_data.xlsx'
